# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [59]:
# Drop unnecessary columns
new_df = df.drop('timestamp', axis=1)

It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [9]:
new_df['rating'].value_counts()

4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: rating, dtype: int64

In [12]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 

reader = Reader(rating_scale=(0.5, 5))
data=Dataset.load_from_df(new_df, reader)


Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [13]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [14]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [27]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run


param_grid = {'n_factors':[10,20, 50, 100],
              'reg_all': [0.05, 0.4, 0.6]}
gs_model = GridSearchCV(SVD,param_grid=param_grid,n_jobs = -1,joblib_verbose=5)
gs_model.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 out of  60 | elapsed:   14.1s remaining:    6.0s
[Parallel(n_jobs=-1)]: Done  55 out of  60 | elapsed:   20.2s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   20.6s finished


In [28]:
# print out optimal parameters for SVD after GridSearch

print(gs_model.best_score)
print(gs_model.best_params)

# {'rmse': 0.8689250510051669, 'mae': 0.6679404366294037}
# {'rmse': {'n_factors': 50, 'reg_all': 0.05}, 'mae': {'n_factors': 100, 'reg_all': 0.05}}

{'rmse': 0.8685746764739948, 'mae': 0.6678525656882434}
{'rmse': {'n_factors': 50, 'reg_all': 0.05}, 'mae': {'n_factors': 50, 'reg_all': 0.05}}


In [36]:
# cross validating with KNNBasic
from surprise.model_selection import train_test_split
np.int = int  # monkey patch deprecated attribute
trainset, testset = train_test_split(data, test_size=0.2)
sim_cos = {"name": "cosine", "user_based": False}
basic = KNNBasic(sim_options=sim_cos)
basic.fit(trainset);

# sim_pearson = {"name": "pearson", "user_based": False}
# basic_pearson = knns.KNNBasic(sim_options=sim_pearson)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [38]:
# print out the average RMSE score for the test set
from surprise import accuracy
predictions = basic.test(testset)
print(accuracy.rmse(predictions))

# ('test_rmse', array([0.97646619, 0.97270627, 0.97874535, 0.97029184, 0.96776748]))
# ('test_mae', array([0.75444119, 0.75251222, 0.7531242 , 0.74938542, 0.75152129]))
# ('fit_time', (0.46678805351257324, 0.54010009765625, 0.7059998512268066, 0.5852491855621338, 1.0139541625976562))
# ('test_time', (2.308177947998047, 2.4834508895874023, 2.6563329696655273, 2.652374029159546, 1.2219891548156738))

RMSE: 0.9758
0.9758029845802755


In [40]:
sim_pearson = {"name": "pearson", "user_based": False}
basic_pearson = KNNBasic(sim_options=sim_pearson)
basic_pearson.fit(trainset)
predictions = basic_pearson.test(testset)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9684
0.968408193810707


In [41]:
# cross validating with KNNBaseline
sim_pearson = {"name": "pearson", "user_based": False}
knn_baseline = KNNBaseline(sim_options=sim_pearson)
knn_baseline.fit(trainset)
predictions = knn_baseline.test(testset)
print(accuracy.rmse(predictions))

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.8802
0.8802179751934307


In [17]:
# print out the average score for the test set



# ('test_rmse', array([0.87268017, 0.88765352, 0.87311917, 0.88706914, 0.87043399]))
# ('test_mae', array([0.66796685, 0.676203  , 0.66790869, 0.67904038, 0.66459155]))
# ('fit_time', (0.6972200870513916, 0.7296440601348877, 0.5842609405517578, 0.609612226486206, 0.61130690574646))
# ('test_time', (1.5466029644012451, 1.567044973373413, 1.6441452503204346, 1.5709199905395508, 1.6216418743133545))

('test_rmse', array([0.87268017, 0.88765352, 0.87311917, 0.88706914, 0.87043399]))
('test_mae', array([0.66796685, 0.676203  , 0.66790869, 0.67904038, 0.66459155]))
('fit_time', (0.6972200870513916, 0.7296440601348877, 0.5842609405517578, 0.609612226486206, 0.61130690574646))
('test_time', (1.5466029644012451, 1.567044973373413, 1.6441452503204346, 1.5709199905395508, 1.6216418743133545))


0.8781911983703239

Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [42]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [43]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [44]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [45]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.0487529529947466, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [46]:
def movie_rater(movie_df,num, genre=None):
    pass
        

In [47]:
# try out the new function here!

If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [124]:
user_rating=[{'userId': 1000, 'movieId': 55245, 'rating': '5'},
 {'userId': 1000, 'movieId': 2491, 'rating': '4'},
 {'userId': 1000, 'movieId': 4718, 'rating': '4'},
 {'userId': 1000, 'movieId': 5990, 'rating': '3'}]
print(df_movies['title'][df_movies['movieId']==55245].item())
print(df_movies['title'][df_movies['movieId']==2491].item())
print(df_movies['title'][df_movies['movieId']==4718].item())
print(df_movies['title'][df_movies['movieId']==5990].item())

Good Luck Chuck (2007)
Simply Irresistible (1999)
American Pie 2 (2001)
Pinocchio (2002)


### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [74]:
## add the new ratings to the original ratings DataFrame

new_new=new_df.append(user_rating,ignore_index=True)
reader = Reader(rating_scale=(0.5, 5))
data2=Dataset.load_from_df(new_new, reader)
dataset2 = data2.build_full_trainset()
print('Number of users: ', dataset2.n_users, '\n')
print('Number of items: ', dataset2.n_items)


Number of users:  611 

Number of items:  9724


In [76]:
# train a model using the new combined DataFrame
trainset2, testset2 = train_test_split(data2, test_size=0.2)

svd = SVD(n_factors=50, reg_all=0.05)
svd.fit(trainset2)
predictions2 = svd.test(testset2)
print(accuracy.rmse(predictions2))

RMSE: 0.8670
0.8669692303975234


In [110]:
df_movies.query("movieId == 1")['title'].item()
# df_movies['title'][df_movies['movieId']==1]

'Toy Story (1995)'

In [115]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
recs = []
for ii in new_new['movieId'].unique():
    recs.append([ii, df_movies['title'][df_movies['movieId']==ii].item(), svd.predict(1000, ii)[3]])

recommendations = pd.DataFrame(recs, columns=['movieId','title','rating'])

In [125]:
# order the predictions from highest to lowest rated

ranked_movies = recommendations.sort_values(by='rating', ascending=False).head(10)
ranked_movies

,movieId,title,rating
1110,1276,Cool Hand Luke (1967),4.573291
335,912,Casablanca (1942),4.524430
830,922,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.524305
232,318,"Shawshank Redemption, The (1994)",4.501648
74,1213,Goodfellas (1990),4.497424
332,904,Rear Window (1954),4.467563
2144,1262,"Great Escape, The (1963)",4.463664
382,2019,Seven Samurai (Shichinin no samurai) (1954),4.452845
722,750,Dr. Strangelove or: How I Learned to Stop Worr...,4.451710
330,899,Singin' in the Rain (1952),4.440890


 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [130]:
user_rating2=[{'userId': 1000, 'movieId': 5545, 'rating': '5'},
 {'userId': 1000, 'movieId': 24491, 'rating': '4'},
 {'userId': 1000, 'movieId': 418, 'rating': '4'},
 {'userId': 1000, 'movieId': 5940, 'rating': '3'}]

In [140]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
    recs = []
    
    new_new=new_df.append(user_ratings,ignore_index=True)
    reader = Reader(rating_scale=(0.5, 5))
    data2=Dataset.load_from_df(new_new, reader)
    dataset2 = data2.build_full_trainset()
    
    svd = SVD(n_factors=50, reg_all=0.05)
    svd.fit(dataset2)
        
    for ii in movie_title_df['movieId'].unique():
        recs.append([ii, movie_title_df['title'][movie_title_df['movieId']==ii].item(), svd.predict(1000, ii)[3]])

    recommendations = pd.DataFrame(recs, columns=['movieId','title','rating']).sort_values(by='rating', ascending=False).head(n)
    return recommendations
            
recommended_movies([],df_movies,10)

,movieId,title,rating
277,318,"Shawshank Redemption, The (1994)",4.379515
906,1204,Lawrence of Arabia (1962),4.360636
602,750,Dr. Strangelove or: How I Learned to Stop Worr...,4.355597
841,1104,"Streetcar Named Desire, A (1951)",4.295697
686,904,Rear Window (1954),4.288654
2226,2959,Fight Club (1999),4.274987
659,858,"Godfather, The (1972)",4.273716
680,898,"Philadelphia Story, The (1940)",4.258934
914,1213,Goodfellas (1990),4.242569
2462,3275,"Boondock Saints, The (2000)",4.241402


In [139]:
user_rating2=[{'userId': 1000, 'movieId': 184791, 'rating': '5'},
 {'userId': 1000, 'movieId': 185435, 'rating': '5'},
 {'userId': 1000, 'movieId': 185473, 'rating': '5'},
 {'userId': 1000, 'movieId': 187593, 'rating': '5'},
 {'userId': 1000, 'movieId': 188189, 'rating': '5'}] 
recommended_movies(user_rating2,df_movies,15)
# df_movies.tail(50)

,movieId,title,rating
602,750,Dr. Strangelove or: How I Learned to Stop Worr...,4.972375
828,1089,Reservoir Dogs (1992),4.958775
277,318,"Shawshank Redemption, The (1994)",4.941588
906,1204,Lawrence of Arabia (1962),4.918786
210,246,Hoop Dreams (1994),4.918635
686,904,Rear Window (1954),4.870154
914,1213,Goodfellas (1990),4.855924
4165,5995,"Pianist, The (2002)",4.842003
659,858,"Godfather, The (1972)",4.832161
6315,48516,"Departed, The (2006)",4.828499


## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.